In [91]:
import pandas as pd
import re
import json
import numpy as np

In [92]:
jets = pd.read_json("jets.json")
details = pd.read_json("jet_details.json")
cleaned_data = pd.DataFrame()

In [93]:
details.head()

,url,image,Crew,Length,Wingspan,Height,Powerplant,Propellers,Maximum speed,Range,...,izdeliye 30,"Climb to 5,000 ft (1,520 m)",Span wings folded,Wing guns,Forward firing guns,Rear cockpit,Anti-radiation missiles,360° turn,Sustained turn time,Upper Wing chord
0,https://wikipedia.org/wiki/AD_Scout,,1,22 ft 9 in (6.93 m),33 ft 5 in (10.19 m),10 ft 3 in (3.12 m),1 × Gnome Monosoupape 9 Type B-2 9-cylinder ai...,2-bladed fixed-pitch propeller,"84 mph (135 km/h, 73 kn) [4]","210 mi (340 km, 180 nmi)[citation needed]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://wikipedia.org/wiki/ACAZ_C.2,https://www.wikipedia.org/wiki/File:ACAZ_C.2_(...,2,8.25 m (27 ft 1 in),12.5 m (41 ft 0 in),3.4 m (11 ft 2 in),1 × Hispano-Suiza 12Ha V-12 water-cooled pisto...,NaN,"250 km/h (160 mph, 130 kn)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://wikipedia.org/wiki/Adamoli-Cattani_fig...,https://www.wikipedia.org/wiki/File:Adamoli-Ca...,1,6.10 m (20 ft 0 in),8.60 m (28 ft 3 in),NaN,1 × le Rhône M 9-cyl. air-cooled rotary piston...,NaN,"186 km/h (116 mph, 100 kn)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://wikipedia.org/wiki/Aerfer_Sagittario_2,https://www.wikipedia.org/wiki/File:Ambrosini_...,1,9.50 m (31 ft 2 in),7.50 m (24 ft 7 in),2.02 m (6 ft 8 in),1 × Rolls-Royce Derwent 9 centrifugal-flow tur...,NaN,"1,006 km/h (625 mph, 543 kn)","765 km (475 mi, 413 nmi)[citation needed]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://wikipedia.org/wiki/AEG_Dr.I,https://www.wikipedia.org/wiki/File:AEG_Dr.I_1...,1,6.1 m (20 ft 0 in),9.4 m (30 ft 10 in),NaN,1 × Mercedes D.IIIa[2] 6-cyl water-cooled in-l...,NaN,"170 km/h (110 mph, 92 kn)",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
nans = details.isna().sum()

In [95]:
one_two_matching = re.compile("((Two)|(One)|(\d+))", re.IGNORECASE)
crew_cleaned = details["Crew"].str.extract(one_two_matching)[0]

one_regex = re.compile("(one)", re.IGNORECASE)
two_regex = re.compile("(two)", re.IGNORECASE)
crew_cleaned = crew_cleaned.replace(one_regex, 1)
crew_cleaned = crew_cleaned.replace(two_regex, 2)

crew_cleaned = pd.to_numeric(crew_cleaned)

cleaned_data["Crew"] = crew_cleaned


In [96]:
meter_regex = re.compile("(((\d+)?.?(\d+))(\s*m))", re.IGNORECASE)

def get_meters(data: pd.Series):
    data = data.str.extract(meter_regex)[1]
    unwanted_chars = re.compile("[(),]")
    data = data.str.replace(unwanted_chars, "")
    data = pd.to_numeric(data)
    return data


In [97]:
meter_columns = details[["Wing area",
                         "Service ceiling",
                         "Rate of climb",
                         "Wingspan",
                         "Height",
                         "Length"]].apply(get_meters)

In [98]:
cleaned_data = pd.concat([cleaned_data, meter_columns], axis=1)

In [99]:
raw_speed = details["Maximum speed"]
speed_extract = re.compile("(((\d+)(.\d+)?)(\s*km/h))", re.IGNORECASE)
mach_extract = re.compile("(mach\s*)((\d+)(\.\d+)?)", re.IGNORECASE)
speed_kmh = raw_speed.str.extract(speed_extract)[1]
speed_kmh = speed_kmh.str.replace(",", "")
highest_speed_extract = re.compile("(\d+)–")
speed_kmh = speed_kmh.str.replace(highest_speed_extract, "")
speed_kmh = pd.to_numeric(speed_kmh)

machs = pd.to_numeric(raw_speed.str.extract(mach_extract)[1])

speed_kmh = speed_kmh.fillna(machs * 1234.8)

cleaned_data["Maximum speed"] = speed_kmh

In [100]:
raw_weight = details[["Empty weight", "Gross weight"]]

weight_regex = re.compile("(((\d+)(.\d+)?)(\s*kg))", re.IGNORECASE)

def get_weight(data: pd.Series):
    data = data.str.extract(weight_regex)[1]
    data = data.str.replace(",", "")
    data = pd.to_numeric(data)
    return data

weight = raw_weight.apply(get_weight)
cleaned_data = pd.concat([cleaned_data, weight], axis=1)


In [101]:
cleaned_jets = pd.DataFrame()


In [102]:
raw_number_created = jets["number created"]
number_extract = re.compile("((\d+)(.\d+)?)")
number_created = raw_number_created.str.extract(number_extract)[0]
number_created = number_created.str.replace(",", "")
number_created = pd.to_numeric(number_created)

cleaned_jets["number created"] = number_created

In [103]:
raw_year = jets["year"]
year_regex = re.compile("(\d+)")
year = raw_year.str.extract(year_regex)[0]
year = pd.to_numeric(year)

cleaned_jets["year"] = year


In [104]:
cleaned_jets = pd.concat([cleaned_jets, jets.drop(["year", "number created"], axis=1)], axis=1)
clean = pd.concat([cleaned_jets, cleaned_data], axis=1)
len(clean)

1303

In [105]:
with open("jet_details.json") as f:
    og_detail_data = json.load(f)
len(og_detail_data)


1229

In [106]:
all_data = []
for item in og_detail_data:
    data_to_add = clean[clean["url"] == item["url"]].to_dict("records")
    if len(data_to_add) > 0:
        item.update(data_to_add[0])
        all_data.append(item)

In [108]:
with open('final_jet_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)